# Demo of the Project<br>

This is a demo for the project which is gonna demonstrate the main functions of the project. <br>
The project is focused on the detection and recognition of traffic signs. <br>
Let's see how it functions.

The first thing we have to do is importing everything that is necessary for the program.

In [ ]:
#Various imports
from Model import ModelCNN as Classifier
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2 as Detector
#from torchvision.models.detection import ssd300_vgg16 as Detector
import matplotlib.pyplot as plt
import torch
from GTSDB_Dataset import GTSDB_Dataset as Dataset
from torch.utils.data import Subset
from __global__  import *
from torchvision.transforms import v2
import cv2 as cv
from TaT_Detection_FastRCNN import test_model
from torchvision.transforms import Resize
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader
import random
from dic_signals import classes



For comfort let's also define these two costants that we will use later. These two constants are used to represent the rows and the columns that we're gonna have in the plots where we display our result.

In [ ]:
# Constants
NUM_SAMPLE_C = 3
NUM_SAMPLE_R = 2

First of all let's setup the device, we prefer the gpu's cuda for efficiency but if it isn't available the only choice is to use the cpu. <br>

We also defined seq, which is a series of trasformations we will apply to the images once they've been opened. The trasformation are mainly for neural network usage, the only thing we are doing is converting to tensor.

In [ ]:
# Prepare images for elaboration
device = ('cuda' if torch.cuda.is_available() else 'cpu') #selecting the device
seq = v2.Compose([
        v2.ToImage(), #converting to tensor image
        v2.ToDtype(torch.float32, scale=True)  
    ]) #trasformation that the images will undergo when used

We have to load the dataset. In this case we load the dataset two times: the first one is loaded with trasformations, so all images will be trasformated according to the instructions in "seq", the second one is raw, so without trasformations. <br>
This because the modified one is used for training and testing, while the second one is used for showing purposes only.

In [ ]:
# Load data
data = Dataset(IMGS_PATH_TEST_GTSDB, transform=seq) #dataset of the images
data_raw = Dataset(IMGS_PATH_TEST_GTSDB) #dataset of the images without trasformation

Once the dataset is load, we have to choose some images so we can do detection and recognition, we choose it randomly. <br>
After choosing the images we load it using the DataLoader.

In [ ]:
# Choose random images

test_images = [] #list of the test images

sub = [random.randint(0, len(data)) for i in range(NUM_SAMPLE_C * NUM_SAMPLE_R)] #random choosing images
dataLoader = DataLoader( Subset(data, sub), batch_size=1, shuffle=False) #declaring dataloader that load subset of dataset images



Before doing the detection and recognition, let's view the images. We show them via plotting to a plot structured in rows and columns. <br>


In [ ]:
# plotting box images

fig = plt.figure(figsize=(128, 200)) #setting the size of the figure to plot to 32x32
rows, cols = NUM_SAMPLE_C, NUM_SAMPLE_R #defining number of columns and rows

y = 0
for i in range(1, rows * cols + 1): 
    
    random_img, name = data_raw.__getitem__(sub[ min(y, NUM_SAMPLE_C * NUM_SAMPLE_R - 1) ]) #getting images
    y = y + 1
    test_images.append(random_img) #appending images to the plot
    fig.add_subplot(rows, cols, i) #
    plt.imshow(random_img) #showing image to the plot
    plt.title(name) #title of image
    plt.axis(False) 
    

About the detection and detection and recognition, we need two neural network, one for the detection and the other for recognition. <br>
So we create these two and load weights into them. We're setting them to evaluation mode and passing to the device.

In [ ]:
# Prepare model models

detector = Detector() #declaring the detector
detector.load_state_dict( 
    torch.load('saved_model/FastRCNN/pesi_ok/model_weights.pth')
    #torch.load('saved_model/ssd300/pesi_ok/model_weights.pth')
) #loading weigths into the model


#declaring the classifier
classifier = Classifier(input_channels=3, input_shape=48, hidden_units=96, output_shape=43)
#loading the weights on the classifier
classifier.load_state_dict(torch.load('saved_model/CNNModel/pesi_ok/model_weights.pth')) 


classifier.eval() #setting the classifier to evaluation mode
detector.eval() #setting the detector to evaluation mode
classifier.to(device) #passing the classifier to device
detector.to(device) #passing the detector to device

print()


Let's define the trasformations that the images will undergo before the detection. <br>
Other than converting to tensor so we can use it in the neural networks, we resize the images to 48x48 and apply some contrast for more clarity.

In [ ]:
bounded = [] #images already done

#trasformation that the image will undergo when opened
pp = v2.Compose(
    [   
        v2.ToImage(), #converting to tensor image
        v2.ToDtype(dtype=torch.float32), #converting to dTyppe
        Resize((48, 48), interpolation=InterpolationMode.NEAREST_EXACT), #resizing image to 48x48
        v2.RandomAutocontrast(p=1.0), #applying contrast   
           
    ]
)

In [ ]:

bounded = [] #images already done

dict_r = test_model(detector, dataLoader, th=0.60, device=device) #doing the test and getting results
for i, d in enumerate(dict_r):
    signals = torch.Tensor([]).to(device) #create tensor and pass it the device
    img = test_images[i]
    
    for box in d['boxes']:
        
        signal = img[box[1]:box[3], box[0]:box[2], :] #getting the sign
        # print( signal.shape)
        signal = pp(signal).unsqueeze(0) #.permute( (2, 0, 1) ) #doing the unsqueeze for the image
        # print( signal.shape)
        signals = torch.cat( (signals, signal.to(device)), dim=0 )
        #signals.append(signal.to(device))


    
    # print("signals :", signals.shape)
    if signals.numel() == 0: #if tensor is empty
        bounded.append(img)
        continue 


    x = classifier(signals) #giving the image to the classifier and getting the result
    signals_dected = x.argmax(dim=1) #getting the max value
    # print(signals_dected.shape)

    for idx, box in enumerate(d['boxes']):
        img = cv.rectangle(img, (box[0], box[1]), (box[2], box[3]), color=(0, 255, 0))#drawing the box around the sign
        #putting the label to the box drawed
        value = signals_dected[idx].cpu().numpy().item()
       
        img = cv.putText(img, f'Label: {classes[value]}', (box[0] - 10, box[1] - 10),  fontFace=cv.FONT_ITALIC, fontScale=0.4, color=(0,255,0))

   
    bounded.append(img) #append the already processed image
    
       
        
 

In [ ]:
fig = plt.figure(figsize=(128, 200)) #defining the size of each figure in the plot
rows, cols = NUM_SAMPLE_C, NUM_SAMPLE_R # number of rows and columns
y = 0
for i in range(1, rows * cols + 1):
    fig.add_subplot(rows, cols, i) #creating the subplot
    plt.imshow(bounded[min(y, len(bounded) - 1)]) #showing the image in the plot
    plt.axis(False)
    y += 1
